In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import pandas as pds

In [3]:
import torch
import torch.nn as nn

In [4]:
rawdata = pds.read_csv("data/energydata_complete.csv")
#rawdata.dtypes

In [5]:
#timestamps are in regular 10 minute intervals, so can model as-if continuous.
data = rawdata.values[:,1:]
numdata = data.shape[0]
order = 3
dim = data.shape[-1]
numsamples = numdata-order+1

sequenced = np.zeros([numsamples, order, dim])
for n in range(0, numsamples):
    begin = n
    end = n+order
    
    sequenced[n]= data[begin:end]


In [6]:
batch_dim = 16

numbatches = round(numsamples/batch_dim)
    
    
batched = np.reshape(sequenced[:batch_dim*numbatches], [numbatches, batch_dim, order, dim])

In [7]:
samples, order, dim = batched[0].shape

testbatch = np.reshape(batched[0], [samples, order*dim])
testbatch = torch.from_numpy(testbatch.astype(np.float32))

In [8]:
class AutoregressionModelLinear(nn.Module):
    
    def __init__(self,dim):
        super(AutoregressionModelLinear, self).__init__()
        
        self.dense = nn.Linear(dim, dim)
        
    def forward(self, x):
        
        return self.dense(x)

In [11]:
model = AutoregressionModelLinear(dim=order*dim)

In [12]:
y = model(testbatch)
y.shape

torch.Size([16, 84])